In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset
from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np



# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
img_size = 480
batch_size = 8
NUM_CLASSES = 257

epochs = 100

In [45]:

# 데이터셋 경로 설정
dataset_path = './256_ObjectCategories'

data_dir = dataset_path


# 데이터 전처리 및 augmentation 설정
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(img_size),
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# ImageFolder를 사용하여 전체 데이터셋 불러오기
#full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
full_dataset = datasets.ImageFolder(data_dir)
# 데이터셋 크기 확인
dataset_size = len(full_dataset)
indices = list(range(dataset_size))
targets = [full_dataset.targets[i] for i in indices]

# StratifiedShuffleSplit을 사용하여 데이터셋 분할
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, val_indices = next(split.split(indices, targets))

# Subset을 사용하여 데이터셋을 분할
train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

train_dataset.dataset.transform = models.EfficientNet_V2_M_Weights.IMAGENET1K_V1.transforms()#data_transforms['train']
val_dataset.dataset.transform = models.EfficientNet_V2_M_Weights.IMAGENET1K_V1.transforms()#data_transforms['val']

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# 클래스 분포 확인
train_classes = [full_dataset.targets[i] for i in train_indices]
val_classes = [full_dataset.targets[i] for i in val_indices]

train_class_distribution = Counter(train_classes)
val_class_distribution = Counter(val_classes)

print(f'Train class distribution: {train_class_distribution}')
print(f'Validation class distribution: {val_class_distribution}')

num_classes = len(train_class_distribution)
print(f'Number of classes: {num_classes}')

Train class distribution: Counter({256: 661, 250: 640, 144: 638, 252: 348, 231: 286, 95: 228, 10: 222, 104: 216, 125: 193, 7: 185, 11: 173, 89: 169, 158: 167, 91: 161, 239: 161, 146: 161, 137: 154, 128: 152, 131: 152, 147: 139, 192: 139, 136: 125, 108: 125, 2: 121, 157: 119, 226: 118, 4: 118, 20: 114, 167: 112, 213: 111, 100: 110, 132: 109, 211: 109, 45: 106, 63: 105, 142: 104, 119: 104, 126: 102, 3: 102, 42: 99, 18: 99, 39: 99, 233: 98, 14: 98, 62: 98, 116: 97, 43: 97, 151: 96, 181: 96, 113: 96, 36: 96, 188: 95, 133: 95, 71: 94, 140: 94, 53: 94, 251: 93, 79: 93, 112: 93, 206: 92, 73: 92, 24: 91, 50: 91, 234: 91, 193: 90, 23: 90, 191: 90, 189: 90, 84: 90, 230: 90, 92: 90, 190: 89, 122: 89, 102: 89, 149: 89, 209: 89, 22: 88, 223: 88, 27: 88, 175: 88, 88: 88, 37: 88, 67: 88, 207: 87, 150: 87, 117: 86, 255: 86, 197: 86, 115: 86, 141: 86, 48: 85, 6: 85, 35: 85, 172: 85, 56: 85, 25: 85, 198: 84, 216: 84, 156: 84, 127: 84, 93: 83, 16: 83, 195: 83, 29: 83, 121: 82, 169: 82, 148: 82, 60: 82, 2

In [46]:
#from efficientnet_pytorch import EfficientNet

model = models.efficientnet_v2_m(weights = models.EfficientNet_V2_M_Weights.IMAGENET1K_V1)
model.classifier = nn.Linear(model.classifier[1].in_features, 512)
model.classifier.add_module("dropout", nn.Dropout(0.4))
model.classifier.add_module("final_fc", nn.Linear(512,num_classes))
model = model.to(device)

In [47]:
import torch.nn.functional as F

class LabelSmoothingCrossEntropy(torch.nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        confidence = 1.0 - self.smoothing
        logprobs = F.log_softmax(pred, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [48]:
warmup_epochs = 1

def lr_lambda(current_epoch):
    if current_epoch < warmup_epochs:
        return float(current_epoch) / float(warmup_epochs)
    else:
        return 0.5 * (1 + np.cos(np.pi * (current_epoch - warmup_epochs) / (epochs - warmup_epochs)))

In [49]:
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [54]:
#criterion = nn.CrossEntropyLoss()
criterion = LabelSmoothingCrossEntropy(smoothing=0.1)

# RAdam, Adam Adagrad S(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

#optimizer = optim.SGD(model.parameters(), lr=0.01)
#optimizer = optim.RAdam(model.parameters(), lr=0.0001)
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True) # val loss 값이 2에폭 동안 작으면 다음 lr을 절반으로
scaler = torch.cuda.amp.GradScaler()  # Mixed Precision Training을 위한 스케일러
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)


#scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=50, T_mult=1, eta_max=0.001,  T_up=2, gamma=0.5)


In [55]:


train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(epochs):
    model.train()
    running_loss, correct = 0.0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct.double() / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc.cpu().numpy())

    model.eval()
    running_loss, correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = correct.double() / len(val_loader.dataset)
    val_losses.append(epoch_loss)
    val_accuracies.append(epoch_acc.cpu().numpy())

    scheduler.step(epoch_loss)

    torch.cuda.empty_cache()

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.4f}')

Validation Epoch 1/100: 100%|██████████| 766/766 [00:45<00:00, 16.91it/s]


Epoch 1/100, Train Loss: 3.2350, Train Acc: 0.4281, Val Loss: 3.2811, Val Acc: 0.5224


Validation Epoch 2/100: 100%|██████████| 766/766 [00:42<00:00, 18.04it/s]


Epoch 2/100, Train Loss: 2.6145, Train Acc: 0.5690, Val Loss: 2.5978, Val Acc: 0.6380


Training Epoch 3/100:  60%|█████▉    | 1833/3061 [05:46<03:52,  5.28it/s]


KeyboardInterrupt: 

: 